# Evaluación Final Módulo 3

In [ ]:
# Tratamiento de datos

import pandas as pd
import numpy as np
import warnings

# Imputación de nulos usando métodos avanzados estadísticos

from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización

import seaborn as sns
import matplotlib.pyplot as plt

# Configuración

pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames
warnings.filterwarnings("ignore")

## Fase 1. Exploración y limpieza

### Exploración csv: 'Customer Flight Activity'

In [ ]:
# Cargar csv 'Customer Flight Activity'
df_cfa = pd.read_csv("data/Customer Flight Activity.csv", index_col=0)
df_cfa.head()

In [ ]:
# Restablecer el índice para convertir 'Loyalty Number' en una columna:
df_cfa.reset_index(inplace=True)
df_cfa.head()

In [ ]:
# Exploración de columnas: 
df_cfa.columns

In [ ]:
# Cantidad de filas y columnas:
print(f"El número de filas que tenemos es {df_cfa.shape[0]}, y el número de columnas es {df_cfa.shape[1]}.")

In [ ]:
# Visualización de la estructura:
df_cfa.info()

In [ ]:
# Visualización estadísticas descriptivas de las columnas numéricas:
df_cfa.describe().T

In [ ]:
# Visualización cantidad de valores únicos de todas las columnas:
df_cfa.nunique()

In [ ]:
# Exploración de los valores únicos de cada columna:

def print_unique_values(df_cfa):
    for column in df_cfa.columns:
        unique_values = df_cfa[column].unique()
        num_unique = len(unique_values)
        print(f"Columna: {column}")
        print(f"Número de valores únicos: {num_unique}")
        print(f"Valores únicos: {unique_values}")
        print("-" * 40)
print_unique_values(df_cfa)

In [ ]:
# Comprobación de Nulos: 
df_cfa.isnull().sum()

In [ ]:
# Función para explorar el número de duplicados por columna: 

def check_duplicates_in_columns(df_cfa):
   
    for column in df_cfa.columns:
        # Identificar los valores duplicados en la columna
        duplicates = df_cfa[column].duplicated(keep=False)
        
        # Contar el número de duplicados en la columna
        num_duplicates = duplicates.sum()
        
        if num_duplicates > 0:
            print(f"Columna '{column}' tiene {num_duplicates} duplicados.")
        else:
            print(f"Columna '{column}' no tiene duplicados.")
check_duplicates_in_columns(df_cfa)

In [ ]:
# Ordenar el DataFrame por 'Loyalty Number'
df_sorted = df_cfa.sort_values(by='Loyalty Number')

# Filtrar las filas duplicadas en 'Loyalty Number' (única columna que a priori podría ser conflictiva en términos de duplicados):
duplicates = df_sorted[df_sorted.duplicated(subset='Loyalty Number', keep=False)]

duplicates.head(20)

!!! Se decide no eliminar duplicados, ya que es necesario mantener los distintos registros de las distintas fechas !!!

### EXPLORACIÓN 'Customer Loyalty History'

In [ ]:
# Cargar csv 'Customer Loyalty History':
df_clh = pd.read_csv("data/Customer Loyalty History.csv")
df_clh.head()

In [ ]:
# Exploración de columnas:
df_clh.columns

In [ ]:
# Cantidad de filas y columnas:
print(f"El número de filas que tenemos es {df_clh.shape[0]}, y el número de columnas es {df_clh.shape[1]}")

In [ ]:
# Visualización de la estructura:

df_clh.info()

In [ ]:
# Visualización estadísticas descriptivas de las columnas numéricas:
df_clh.describe().T

In [ ]:
# Visualización estadísticas descriptivas de las columnas categóricas o tipo object:
df_clh.describe(include="object").T

In [ ]:
# Visualicación cantidad de valores únicos de cada columna:
df_clh.nunique()

In [ ]:
# Función para explorar los valores únicos de cada columna:

def print_unique_values(df_clh):
    for column in df_clh.columns:
        unique_values = df_clh[column].unique()
        num_unique = len(unique_values)
        print(f"Columna: {column}")
        print(f"Número de valores únicos: {num_unique}")
        print(f"Valores únicos: {unique_values}")
        print("-" * 40)
print_unique_values(df_clh)

In [ ]:
# Comprobación de nulos:
df_clh.isnull().sum()

In [ ]:
# Comprobar que en la columna Loyalty Number (única columna que podría ser conflictiva en términos de duplicados) todos los valores son únicos:
num_duplicates = df_clh['Loyalty Number'].duplicated().sum()
print(f"Número de valores duplicados en 'Loyalty Number': {num_duplicates}")

## 1.2. Transformación y Limpieza: 

In [ ]:
# Unir las columnas 'Year' y 'Month' de 'df_cfa' en una columna de tipo datetime: 

df_cfa['Date'] = pd.to_datetime(df_cfa['Year'].astype(str) + '-' + df_cfa['Month'].astype(str) + '-01')

# Eliminar las columnas originales de 'Year' y 'Month':

df_cfa.drop(columns=['Year', 'Month'], inplace=True)

In [ ]:
# Función para unir columnas del 'df_clh' en una columna de tipo date time y eliminar las originales:

def create_date_columns(df_clh, enrollment_year_col, enrollment_month_col, cancellation_year_col, cancellation_month_col):
    df_clh['Enrollment Date'] = pd.to_datetime(df_clh[enrollment_year_col].astype(str) + '-' + df_clh[enrollment_month_col].astype(str) + '-01', errors='coerce')
    df_clh['Cancellation Date'] = pd.to_datetime(df_clh[cancellation_year_col].astype('Int64').astype(str) + '-' + df_clh[cancellation_month_col].astype('Int64').astype(str) + '-01', errors='coerce')
    df_clh.drop(columns=[enrollment_year_col, enrollment_month_col, cancellation_year_col, cancellation_month_col], inplace=True)

    return df_clh

df_clh = create_date_columns(df_clh, 'Enrollment Year', 'Enrollment Month', 'Cancellation Year', 'Cancellation Month')

In [ ]:
# Cambiar tipo de dato de columnas de 'df_cfa':

columns_to_convert = ['Distance', 'Points Accumulated', 'Points Redeemed', 'Dollar Cost Points Redeemed']
df_cfa[columns_to_convert] = df_cfa[columns_to_convert].astype(float)

In [ ]:
# Cambiar tipo de dato de las columnas de df_clh:

categorical_columns = ['Country', 'Province', 'City', 'Gender', 'Education', 'Marital Status', 'Loyalty Card', 'Enrollment Type']
df_clh[categorical_columns] = df_clh[categorical_columns].astype('category')

In [ ]:
# Calcular los porcentajes de nulos para valorar la imputación:

total_rows =len(df_clh)
null_counts = df_clh.isnull().sum()
null_counts_filtered = null_counts[null_counts > 0]
null_percentages = (null_counts_filtered/ total_rows) * 100
null_percentages

In [ ]:
# Filtra las filas donde 'Salary' es nulo y contar la cantidad de valores nulos en 'Salary' por cada categoría de 'Education':

nulos_salary = df_clh[df_clh['Salary'].isnull()]
categoria_nulos = nulos_salary['Education'].value_counts()

print("Categorías de 'Education' con valores nulos en 'Salary':")
print(categoria_nulos)

In [ ]:
# Imputar con 0 los nulos de la columna "Salary", ya que corresponden a la categoría "College" de la columna "Education" y se entiende que no tienen salario: 

df_clh.loc[df_clh['Education'] == 'College', 'Salary'] = df_clh.loc[df_clh['Education'] == 'College', 'Salary'].fillna(0)


In [ ]:
#Calcular la cantidad de valores negativos en la columna "Salary":

negative_count = (df_clh['Salary'] < 0).sum()
negative_count

In [ ]:
# Mostrar los registros con valores negativos en la columna "Salary":

negative_salaries = df_clh[df_clh['Salary'] < 0]
negative_salaries

In [ ]:
# Convertir los valores negativos a positivos usando numpy (Por el tipo de valores que se observan, se interpreta que ha habido un problema de introducción de datos):

df_clh['Salary'] = np.abs(df_clh['Salary'])

#----> También se podría hacer con apply: df_clh['Salary'] = df_clh['Salary'].apply(lambda x: abs(x))

In [ ]:
# Convertir los valores nulos de la columna "Cancellation Date" a "-", ya que se interpreta que sigue vigente:

df_clh['Cancellation Date'] = df_clh['Cancellation Date'].fillna('-')
print("Número de nulos en 'Cancellation Date' después de imputación:", df_clh['Cancellation Date'].isnull().sum())

In [ ]:
# Unión de los csv:

df_cfalh = pd.merge(df_cfa, df_clh, on='Loyalty Number', how='inner')
df_cfalh.head()

## FASE 2. Visualización

2.1. ¿Cómo se distribuye la cantidad de vuelos reservados por mes durante el año?

In [ ]:
# Crear una columna 'Year-Month' para el análisis mensual:

df_cfalh['Year-Month'] = df_cfalh['Date'].dt.to_period('M') 

# Agrupar por mes y sumar la cantidad de vuelos reservados:

monthly_flights = df_cfalh.groupby('Year-Month')['Flights Booked'].sum().reset_index()

# Gráfico:

plt.figure(figsize=(10, 4))

plt.plot(monthly_flights['Year-Month'].astype(str), monthly_flights['Flights Booked'], marker='o', linestyle='-')

plt.xlabel('Año-Mes')
plt.ylabel('Vuelos reservados')
plt.title('Distribución de Vuelos Reservados por Mes durante el Año')
plt.xticks(rotation=45)  # Rotar etiquetas del eje x para mejor visibilidad
plt.grid(True)

plt.tight_layout() # Márgenes
plt.show()


- Estacionalidad: Los picos de vuelos reservados en los meses de verano y Navidad indican un mayor número de reservas durante ciertos períodos del año que implican vacaciones o festividades.

2.2. ¿Existe una relación entre la distancia de los vuelos y los puntos acumulados por los clientes?

In [ ]:
# Calcular la correlación de Pearson entre distancia y puntos acumulados:

correlation = df_cfalh['Distance'].corr(df_cfalh['Points Accumulated'])

print(f'Correlación entre Distance y Points Accumulated: {correlation:.2f}')

In [ ]:
plt.figure(figsize=(10, 6))

# Crear un gráfico de dispersión con color por la categoría Loyalty Card:

sns.scatterplot(data=df_cfalh, x='Distance', y='Points Accumulated', hue='Loyalty Card')

plt.xlabel('Distancia')
plt.ylabel('Puntos Acumulados')
plt.title('Relación entre Distancia de los Vuelos y Puntos Acumulados por Categoría')
plt.legend(title='Targeta de Fidelización')
plt.grid(True)

plt.show()


- Relación positiva: las diferentes categorías de tarjetas pueden mostrar variaciones en la cantidad de puntos acumulados por distancia, pero todas muestran una tendencia general de mayor acumulación de puntos con una mayor distancia. 
- Esto sugiere que hay una diferencia en la generosidad de las recompensas entre los tipos de tarjeta.


3. ¿Cuál es la distribución de los clientes por provincia o estado?

In [ ]:
# Número de clientes por Provincia
provincia_counts = df_cfalh['Province'].value_counts()

# Convertir a DataFrame para facilitar la visualización
provincia_df = provincia_counts.reset_index()
provincia_df.columns = ['Province', 'Number of Clients']

# Gráfico:
plt.figure(figsize=(10, 6))

sns.barplot(data=provincia_df, x='Province', y='Number of Clients', palette='viridis')
plt.xlabel('Provincia')
plt.ylabel('Número de Clientes')
plt.title('Distribución de Clientes por Provincia')
plt.xticks(rotation=45)  # Rotar etiquetas del eje 
plt.grid(True)

plt.show()

- El gráfico simplemente revela las provincias con mayor número de clientes. 
- Las provincias con un alto número de clientes podrían ser el foco para estrategias de marketing y servicio, mientras que provincias con menos clientes podrían necesitar esfuerzos adicionales para aumentar la base de clientes.

4. ¿Cómo se compara el salario promedio entre los diferentes niveles educativos de los clientes?

In [ ]:
# Salario promedio por nivel educativo
salary_by_education = df_cfalh.groupby('Education')['Salary'].mean().reset_index()

# Renombrar las columnas
salary_by_education.columns = ['Education', 'Average Salary']

# Gráfico:
plt.figure(figsize=(10, 6))

sns.barplot(data=salary_by_education, x='Education', y='Average Salary', palette='viridis')

plt.xlabel('Nivel Educativo')
plt.ylabel('Salario Promedio')
plt.title('Salario Promedio por Nivel Educativo')
plt.xticks(rotation=45) 
plt.grid(True)

plt.show()

In [ ]:
# Boxplot:

df_filtered = df_cfalh[df_cfalh['Education'] != 'College']

plt.figure(figsize=(12, 8))

sns.boxplot(data=df_filtered, x='Education', y='Salary', palette='Set2')

plt.xlabel('Nivel Educativo', fontsize=14, fontweight='bold')
plt.ylabel('Salario', fontsize=14, fontweight='bold')
plt.title('Distribución de Salarios por Nivel Educativo (sin College)', fontsize=16, fontweight='bold')
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

- Se observan diferencias significativas entre los salarios promedio de diferentes niveles educativos, reflejando cómo un nivel más alto de educación (Doctor) influye positivamente en el nivel de ingresos.
- Utilizando el boxplot se pueden apreciar los outliers que corresponden a los salarios atípicos más bajos en el caso de "Bachelor" y "High School or Below", así como los outliers con valores que representan salarios atípicos más altos, como es el caso de la categoría "Doctor". 

5. ¿Cuál es la proporción de clientes con diferentes tipos de tarjetas de fidelidad?

In [ ]:
# Número de clientes por tipo de tarjeta
loyalty_card_counts = df_cfalh['Loyalty Card'].value_counts()

# Calcular las proporciones
loyalty_card_proportions = loyalty_card_counts / loyalty_card_counts.sum()

# Gráfico
plt.figure(figsize=(8, 8))

plt.pie(loyalty_card_proportions, labels=loyalty_card_proportions.index, autopct='%1.1f%%', colors=['#ff9999','#66b3ff','#99ff99','#ffcc99'])
plt.title('Proporción de Clientes por Tipo de Tarjeta de Fidelidad')

plt.show()

- Se puede identificar que la targeta Star tiene la mayor proporción de clientes. 
- Como se puede comprobar en el gráfico 'Relación entre Distancia de los Vuelos y Puntos Acumulados por Categoría', seguramente es debido a que es la targeta con un coste más bajo. 
- A priori se podría concluir que los beneficios de la targeta Aurora no compensan su precio, en relación con las targetas de menor categoría.

6. ¿ Cómo se distribuyen los clientes según su estado civil y género?

In [ ]:
# Crear una tabla de contingencia
contingency_table = pd.crosstab(index=df_cfalh['Marital Status'], columns=df_cfalh['Gender'])

# Mostrar la tabla de contingencia
print("Tabla de Contingencia:")
print(contingency_table)

# Gráfico
plt.figure(figsize=(10, 6))

contingency_table.plot(kind='bar', figsize=(12, 8))

plt.xlabel('Estado Civil')
plt.ylabel('Número de Clientes')
plt.title('Distribución de Clientes según Estado Civil y Género')
plt.legend(title='Género')
plt.grid(True)

plt.show()



In [ ]:
# Heatmap:

contingency_table = pd.crosstab(index=df_cfalh['Marital Status'], columns=df_cfalh['Gender'])

plt.figure(figsize=(12, 8))

sns.heatmap(contingency_table, annot=True, cmap='YlGnBu', fmt='d')

plt.xlabel('Género')
plt.ylabel('Estado Civil')
plt.title('Distribución de Clientes según Estado Civil y Género')
plt.show()


- Se observa como no hay diferencias significativas entre géneros en las distintas categorías. 
- El mayor número de clientes corresponde a la categoría de casados, mientras que el menor número de clientes corresponde a la categoría de divorciados. 
- La diferencia de cantidades de clientes entre categorías es significativa, sobretodo para la categoría de casados. Esto podría indicar que las personas casadas tienden a contratar programas de fidelización. 

## FASE 3. Evaluación de Diferencias en Reservas de Vuelos por Nivel Educativo

- Hipótesis:

    - H0​: No hay una diferencia significativa en el número promedio de vuelos reservados entre los grupos de "Secondary Education" y "Higher Education".
    - H1​: Hay una diferencia significativa en el número promedio de vuelos reservados entre los grupos de "Secondary Education" y "Higher Education".

- Preparación de los datos:

In [ ]:
# Agrupar niveles educativos
group_a = ['High School', 'College']  # Educación secundaria
group_b = ['Bachelor', 'Master', 'Doctorate']  # Educación superior

# Crear una nueva columna para el grupo
df_cfalh['Education Group'] = df_cfalh['Education'].apply(lambda x: 'Secondary Education' if x in group_a else 'Higher Education')

# Filtrar el DataFrame para incluir solo las columnas relevantes
df_ab_test = df_cfalh[['Flights Booked', 'Education Group']]

- Visualización estadísticas descriptivas de ambos grupos:

In [ ]:
# Calcular estadísticas descriptivas por grupo de educación
descriptive_stats_ab = df_ab_test.groupby('Education Group')['Flights Booked'].describe()
descriptive_stats_ab

- Pruebas de Normalidad:

In [ ]:
from scipy.stats import shapiro

# Filtrar los datos por grupo
group_a_data = df_ab_test[df_ab_test['Education Group'] == 'Secondary Education']['Flights Booked']
group_b_data = df_ab_test[df_ab_test['Education Group'] == 'Higher Education']['Flights Booked']

# Valor predeterminado de alpha
alpha = 0.05

# Prueba de normalidad para el Grupo A
stat_a, p_value_a = shapiro(group_a_data)
normality_a = "normal" if p_value_a > alpha else "no normal"
print(f"Grupo A - Prueba de Normalidad Shapiro-Wilk p-value: {p_value_a:.4e} (Resultado: {normality_a})")

# Prueba de normalidad para el Grupo B
stat_b, p_value_b = shapiro(group_b_data)
normality_b = "normal" if p_value_b > alpha else "no normal"
print(f"Grupo B - Prueba de Normalidad Shapiro-Wilk p-value: {p_value_b:.4e} (Resultado: {normality_b})")


- Visualización distribución no normal:

In [ ]:
plt.figure(figsize=(12, 6))

# Histograma y curva de densidad para el Grupo A
sns.histplot(df_ab_test[df_ab_test['Education Group'] == 'Secondary Education']['Flights Booked'], kde=True, label='Secondary Education', color='blue', alpha=0.6)
sns.histplot(df_ab_test[df_ab_test['Education Group'] == 'Higher Education']['Flights Booked'], kde=True, label='Higher Education', color='orange', alpha=0.6)

# Configurar etiquetas y título
plt.xlabel('Vuelos Reservados')
plt.ylabel('Frecuencia')
plt.title('Histograma con Curva de Densidad de Vuelos Reservados por Nivel Educativo')
plt.legend()
plt.grid(True)

# Mostrar el gráfico
plt.show()

- Como la distribución de datos no es normal, se utiliza la prueba no paramétrica de Mann-Whitney U:

In [ ]:
from scipy.stats import mannwhitneyu

# Prueba de Mann-Whitney U
u_stat, p_value = mannwhitneyu(group_a_data, group_b_data, alternative='two-sided')
print(f"Prueba Mann-Whitney U - Estadístico U: {u_stat:.2f}, p-value: {p_value:.2f}")

# Evaluar si rechazar H0
if p_value < alpha:
    print("Rechazamos la hipótesis nula. Existe una diferencia significativa en el número de vuelos reservados entre los dos grupos.")
else:
    print("No rechazamos la hipótesis nula. No hay evidencia suficiente para afirmar que existe una diferencia significativa en el número de vuelos reservados entre los dos grupos.")


### CONCLUSIÓN
- Los resultados de la prueba Mann-Whitney U muestran un estadístico U de 15,631,115,821.00 y un valor p de 0.01. 
- Dado que el valor p es menor que el nivel de significancia (α = 0.05), se rechaza la hipótesis nula, indicando que existe una diferencia estadísticamente significativa en el número de vuelos reservados entre los dos grupos comparados. 
- Los datos obtenidos sugieren que las diferencias en el número de vuelos reservados no se deben al azar y que, en efecto, hay una discrepancia significativa en la conducta de reserva entre los grupos evaluados.